## I referred to the following webpages for the implementation.
- Implementation of Transformer<br>
https://qiita.com/halhorn/items/c91497522be27bde17ce<br>
https://github.com/kpot/keras-transformer/tree/master/keras_transformer<br>
https://github.com/Lsdefine/attention-is-all-you-need-keras<br>
- Usage of "\_\_call\_\_" method<br>
https://qiita.com/kyo-bad/items/439d8cc3a0424c45214a

In [1]:
import numpy as np
import math

import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Layer, Embedding, Input, Reshape, Lambda, Add
from keras import backend as K
from keras.initializers import RandomNormal
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import Callback

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

In [2]:
vocab_size = 8000
d_model = 512
MAX_LEN = 716
class_num = 9
PAD_ID = 0
warmup_steps = 4000
NUM_TRAIN = 5893
NUM_TEST = 1474
batch_size = 16
epochs = 30

In [3]:
def parse(example):
    features = tf.parse_single_example(
        example,
        features={
            "X": tf.FixedLenFeature([MAX_LEN], dtype=tf.float32),
            "Y": tf.FixedLenFeature((class_num,), dtype=tf.float32)
        }
    )
    
    X = features["X"]
    Y = features["Y"]
    
    return X, Y

In [4]:
def iterator(tfrecord_path, data_len):
    dataset = tf.data.TFRecordDataset([tfrecord_path]).map(parse)
    dataset = dataset.repeat(-1).batch(data_len)
    iterator = dataset.make_one_shot_iterator()
    
    X, Y = iterator.get_next()
    X = tf.reshape(X, [-1, MAX_LEN])
    
    return X, Y

In [5]:
%%time

x_train, y_train = iterator("train_transformer_with_sp.tfrecord", NUM_TRAIN)
x_test, y_test = iterator("test_transformer_with_sp.tfrecord", NUM_TEST)

x_train = tf.Session().run(x_train)
y_train = tf.Session().run(y_train)
x_test = tf.Session().run(x_test)
y_test = tf.Session().run(y_test)

CPU times: user 2.1 s, sys: 1.17 s, total: 3.27 s
Wall time: 2.45 s


In [6]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5893, 716), (5893, 9), (1474, 716), (1474, 9))

## Transformer Architecture

In [7]:
class MultiheadAttention():
    ## hidden_dim has to be multiples of head_num
    def __init__(self, max_len, hidden_dim=512, head_num=8, dropout_rate=0.1, *args, **kwargs):
        self.max_len = max_len
        self.hidden_dim = hidden_dim
        self.head_num = head_num
        self.dropout_rate = dropout_rate
        
        self.q_dense_layer = Dense(hidden_dim, use_bias=False)
        self.k_dense_layer = Dense(hidden_dim, use_bias=False)
        self.v_dense_layer = Dense(hidden_dim, use_bias=False)
        self.output_dense_layer = Dense(hidden_dim, use_bias=False)
        self.attention_dropout_layer = Dropout(dropout_rate)
        
    def split_heads(self, x):
        def reshape(x):
            x = tf.reshape(x, [-1, self.max_len, self.head_num, self.hidden_dim // self.head_num])
            return tf.transpose(x, [0, 2, 1, 3])
        
        out = Lambda(reshape)(x)
        return out
    
    def combine_heads(self, heads):
        def reshape(x):
            heads = tf.transpose(x, [0, 2, 1, 3])
            return tf.reshape(x, [-1, self.max_len, self.hidden_dim])
        
        out = Lambda(reshape)(heads)
        return out
        
    def __call__(self, query, memory):
        #two arguments of query and memory are already encoded as embedded vectors for all words
        q = self.q_dense_layer(query)
        k = self.k_dense_layer(memory)
        v = self.v_dense_layer(memory)
        
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        #for scaled dot-product
        depth_inside_each_head = self.hidden_dim // self.head_num
        q = Lambda(lambda x: x * (depth_inside_each_head ** -0.5))(q)
        
        #q.shape = (batch_size, head_num, query_len, emb_dim)
        #k.shape = (batch_size, head_num, memory_len, emb_dim)
        #batch_dot(q, k).shape = (batch_size, head_num, query_len, memory_len)
        score = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[3, 3]))([q, k])
        normalized_score = Activation("softmax")(score)
        normalized_score = self.attention_dropout_layer(normalized_score)
        
        #normalized_score.shape = (batch_size, head_num, query_length, memory_length)
        #v.shape = (batch_size, head_num, memory_length, depth)
        #attention_weighted_output.shape = (batch_size, head_num, query_length, depth)
        attention_weighted_output = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[3, 2]))([normalized_score, v])
        attention_weighted_output = self.combine_heads(attention_weighted_output)
        return self.output_dense_layer(attention_weighted_output)

In [8]:
# SlefAttention class inherits MultiheadAttention class so that it can make query and memory come from the same source.
class SelfAttention(MultiheadAttention):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def __call__(self, query):
        return super().__call__(query, query)

In [9]:
class PositionwiseFeedForwardNetwork():
    
    def __init__(self, hidden_dim, dropout_rate, *args, **kwargs):
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        
        self.first_dense_layer = Dense(hidden_dim*4, use_bias=True, activation="relu")
        self.second_dense_layer = Dense(hidden_dim, use_bias=True, activation="linear")
        self.dropout_layer = Dropout(dropout_rate)
        
    def __call__(self, inputs):
        # make the network more flexible to learn for the first dense layer(non-linear transformation is used),
        # and put the network back into the same hidden dim as original(linear transformation is used)
        x = self.first_dense_layer(inputs)
        x = self.dropout_layer(x)
        return self.second_dense_layer(x)

In [10]:
class LayerNormalization(Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis
        super(LayerNormalization, self).__init__(**kwargs)
        
    def get_config(self):
        config = super().get_config()
        config["axis"] = self.axis
        return config
    
    def build(self, input_shape):
        hidden_dim = input_shape[-1]
        self.scale = self.add_weight("layer_norm_scale", shape=[hidden_dim],
                                    initializer="ones")
        self.shift = self.add_weight("layer_norm_shift", shape=[hidden_dim],
                                    initializer="zeros")
        super(LayerNormalization, self).build(input_shape)
        
    def call(self, inputs, epsilon=1e-6):
        mean = K.mean(inputs, axis=[-1], keepdims=True)
        variance = K.var(inputs, axis=[-1], keepdims=True)
        normalized_inputs = (inputs - mean) / (K.sqrt(variance) + epsilon)
        return normalized_inputs * self.scale + self.shift
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [11]:
class PreLayerNormPostResidualConnectionWrapper():
    def __init__(self, layer, dropout_rate, *args, **kwargs):
        self.layer = layer
        self.layer_norm = LayerNormalization()
        self.dropout_layer = Dropout(dropout_rate)
        
    def __call__(self, inputs, *args, **kwargs):
        x = self.layer_norm(inputs)
        x = self.layer(x)
        outputs = self.dropout_layer(x)
        results = Add()([inputs, outputs])
        return results

In [12]:
class AddPositionalEncoding(Layer): 
    def call(self, inputs):
        data_type = inputs.dtype
        batch_size, max_len, emb_dim = tf.unstack(tf.shape(inputs))
        # i is from 0 to 255 when emb_dim is 512
        #so the doubled_i is from 0 to 510
        doubled_i = K.arange(emb_dim) // 2 * 2
        exponent = K.tile(K.expand_dims(doubled_i, 0), [max_len, 1])
        denominator_matrix = K.pow(10000.0, K.cast(exponent / emb_dim, data_type))
        
        # since cos(x) = sin(x + π/2), we convert the series of [sin, cos, sin, cos, ...]
        # into [sin, sin, sin, sin, ...]
        to_convert = K.cast(K.arange(emb_dim) % 2, data_type) * math.pi / 2
        convert_matrix = K.tile(tf.expand_dims(to_convert, 0), [max_len, 1])
        
        seq_pos = K.arange(max_len)
        numerator_matrix = K.cast(K.tile(K.expand_dims(seq_pos, 1), [1, emb_dim]), data_type)
        
        positinal_encoding = K.sin(numerator_matrix / denominator_matrix + convert_matrix)
        batched_positional_encoding = K.tile(K.expand_dims(positinal_encoding, 0), [batch_size, 1, 1])
        return inputs + batched_positional_encoding
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [13]:
class MakeZeroPads(Layer):
    def __init__(self, seq_len, vocab_size, emb_dim, data_type="float32", *args, **kwargs):
        self.emb_dim = emb_dim
        super(MakeZeroPads, self).__init__(*args, **kwargs)
        
    def call(self, inputs):
        mask_for_pads = tf.to_float(tf.not_equal(inputs, PAD_ID))
        pads_masked_embedding = inputs * mask_for_pads
        return pads_masked_embedding * (self.emb_dim ** 0.5)
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [14]:
class Encoder():
    def __init__(self, vocab_size, max_len, stack_num, head_num, emb_dim, dropout_rate, *args, **kwargs):
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.stack_num = stack_num
        self.head_num = head_num
        self.emb_dim = emb_dim
        self.dropout_rate = dropout_rate
        
        self.embedding_layer = Embedding(self.vocab_size,
                           self.emb_dim,
                           embeddings_initializer=RandomNormal(mean=0.0, stddev=self.emb_dim**-0.5)
                          )
        self.make_zero_pads_layer = MakeZeroPads(self.max_len, vocab_size, emb_dim)
        self.add_pos_enc_layer = AddPositionalEncoding()
        self.input_dropout_layer = Dropout(dropout_rate)
        
        self.attention_block_list = []
        for _ in range(stack_num):
            self_attention_layer = SelfAttention(self.max_len, self.emb_dim, self.head_num, self.dropout_rate)
            pffn_layer = PositionwiseFeedForwardNetwork(self.emb_dim, self.dropout_rate)
            self.attention_block_list.append([
                PreLayerNormPostResidualConnectionWrapper(self_attention_layer, dropout_rate),
                PreLayerNormPostResidualConnectionWrapper(pffn_layer, dropout_rate)
            ])
        self.output_layer_norm = LayerNormalization()
        
    def __call__(self, inputs):
        x = self.embedding_layer(inputs)
        x = self.make_zero_pads_layer(x)
        x = self.add_pos_enc_layer(x)
        x = self.input_dropout_layer(x)
        
        for i, set_of_layers_list in enumerate(self.attention_block_list):
            self_attention_layer, pffn_layer = tuple(set_of_layers_list)
            x = self_attention_layer(x)
            x = pffn_layer(x)
            
        return self.output_layer_norm(x)

In [15]:
# Transformer classification model
inputs = Input((MAX_LEN,))
transformer_encoder = Encoder(vocab_size=vocab_size, stack_num=6, head_num=8, emb_dim=512, dropout_rate=0.1, max_len=MAX_LEN)
encoder_output = transformer_encoder(inputs)
#Since the task is text classification, we just need the first token for each sample
summarized_vecs = Lambda(lambda x: x[:, 0, :])(encoder_output)
outputs = Dense(class_num)(summarized_vecs)
model = Model(inputs, outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 716)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 716, 512)     4096000     input_1[0][0]                    
__________________________________________________________________________________________________
make_zero_pads_1 (MakeZeroPads) (None, 716, 512)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
add_positional_encoding_1 (AddP (None, 716, 512)     0           make_zero_pads_1[0][0]           
__________________________________________________________________________________________________
dropout_1 

In [16]:
plot_model(model, to_file="transformer_encoder.png", show_shapes=True, show_layer_names=True)

In [17]:
#learning rate for Adam
class LRSchedulerPerStep(Callback):
    def __init__(self, d_model, warmup_steps=4000):
        self.d_model = d_model
        self.warmup_step = warmup_steps
        self.step_num = 0
        
    def on_batch_begin(self, batch, logs=None):
        self.step_num += 1
        updated_lr = self.d_model * min(self.step_num ** (-0.5), self.step_num * (self.warmup_step ** (-1.5)))
        K.set_value(self.model.optimizer.lr, updated_lr)

In [18]:
lr_scheduler = LRSchedulerPerStep(d_model, warmup_steps)

In [19]:
opt = Adam(beta_1=0.9, beta_2=0.98, epsilon=1e-9)

## About customized loss function
https://stackoverflow.com/questions/50063613/add-loss-function-in-keras<br>
https://github.com/kpot/keras-transformer/blob/b9d4e76c535c0c62cadc73e37416e4dc18b635ca/keras_transformer/bert.py#L212<br>
https://github.com/tensorflow/models/blob/b9ef963d1e84da0bb9c0a6039457c39a699ea149/official/transformer/v2/metrics.py#L47<br>

In [20]:
# smoothed loss function
class SmoothedLossSparseCategoricalXEntropy:
    def __init__(self, smoothing, class_num):
        self.smoothing = smoothing
        self.class_num = class_num
        
    def __call__(self, y_true, y_pred):
        confidence = 1.0 - self.smoothing
        low_confidence = (1.0 - confidence) / tf.cast(self.class_num - 1, tf.float32)
        smoothed_labels = tf.one_hot(
            tf.cast(y_true, tf.int32),
            depth=self.class_num,
            on_value=confidence,
            off_value=low_confidence
        )
        xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=y_pred,
            labels=smoothed_labels
        )
        
        lowest_loss = -(confidence * tf.log(confidence) + 
                       tf.cast(self.class_num -1, tf.float32) * low_confidence * tf.log(low_confidence + 1e-20))
        final_loss = xentropy - lowest_loss
        return final_loss

In [21]:
y_train = np.argmax(y_train, axis=-1)
y_test = np.argmax(y_test, axis=-1)
y_train.shape, y_test.shape

((5893,), (1474,))

In [22]:
model.compile(optimizer=opt, loss=SmoothedLossSparseCategoricalXEntropy(smoothing=0.1, class_num=class_num))

In [25]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[lr_scheduler])

Train on 5893 samples, validate on 1474 samples
Epoch 1/30
5893/5893 [==============================] - 133s 23ms/step - loss: 974.9758 - val_loss: 2331.3364
Epoch 2/30
5893/5893 [==============================] - 133s 23ms/step - loss: 965.6583 - val_loss: 844.5497
Epoch 3/30
5893/5893 [==============================] - 133s 23ms/step - loss: 919.7374 - val_loss: 2032.6272
Epoch 4/30
5893/5893 [==============================] - 133s 23ms/step - loss: 902.4449 - val_loss: 618.2776
Epoch 5/30
5893/5893 [==============================] - 133s 23ms/step - loss: 829.1413 - val_loss: 804.5805
Epoch 6/30
5893/5893 [==============================] - 133s 23ms/step - loss: 842.8073 - val_loss: 1510.1228
Epoch 7/30
5893/5893 [==============================] - 133s 23ms/step - loss: 884.1554 - val_loss: 1658.1582
Epoch 8/30
5893/5893 [==============================] - 133s 23ms/step - loss: 777.7955 - val_loss: 935.3201
Epoch 9/30
5893/5893 [==============================] - 133s 23ms/step - los